In [1]:
import os
import numpy as np
import argparse

import h5py
from sklearn.cluster import KMeans
from matplotlib import cm
from PIL import Image
import openslide

import torch
import torchvision
from torchvision import transforms
import torch.nn as nn

# # Kmeans analysis of BYOL derived latent space
In this notebook I will perform kmeans clustering on feature vectors produced by BYOL. The dataset is AT8 immunohistochemically stained whole slide images of human hippocampus that have been digitized. The kmeans clustering will show how richely the features represent the anatomy and pathology of the dataset. 

The below code is a Pytorch dataloader which yields a tensor of a 256x256 patch of the provided slide image at the index of coordinates. The function uses an h5file which is the ouput of a segmentation function provided by CLAM (https://github.com/mahmoodlab/CLAM/blob/master/create_patches.py) which is an array of top-left coordinates of 256x256 patches that contain tissue. This ensures that we do not get any background in our analysis.  

In [5]:
class FeatureCreationDataset(torch.utils.data.Dataset):
    """ This dataset assumes that only one slide will be instantiated with.
        Additionally, it assumes no transforms.
    """
    def __init__(self, slide='', h5file='', tilesize = 256):
        self.tilesize = tilesize
        
        self.slidepath = slide
        self.wsi = openslide.OpenSlide(slide)
        
        self.h5path = h5file
        f = h5py.File(h5file,'r')
        self.coords = np.asarray(f['coords'])

        self.transform = transforms.ToTensor()

    def __len__(self):
        return(self.coords.shape[0])
    
    def __getitem__(self, idx):
        coord = self.coords[idx,:]
        tile = self.wsi.read_region((coord[0], coord[1]),0,(self.tilesize, self.tilesize)).convert('RGB')

        # img = img_as_float32(img)
        img = self.transform(tile)
            
        return img

This function will create the feature vectors based on the supplied slide images using a supplied pretrained BYOL model. It can also create feature vectors using Resnet50 pretrained on an Imagenet classification task (weights provided by Pytorch).

In [ ]:
def make_byol_features_to_np(slide='', h5file='', checkpoint="/sc/arion/projects/tauomics/BYOL-tau-scoring/BYOL/models/trained_resnet6.pth", imagenet_pretrained=False, tilesize = 256, batch_size = 400):
    """ Make features from images by passing them through a resnet encoder"""
    
    tile_dataset = FeatureCreationDataset(slide=slide, h5file=h5file, tilesize = 256)
    loader = torch.utils.data.DataLoader(tile_dataset,
                                              batch_size=batch_size, 
                                              num_workers=5, 
                                              shuffle=False,
                                              drop_last=False)

    device=torch.device('cuda')
    
    # encoder setup
    if imagenet_pretrained:
        resnet = torchvision.models.resnet50(pretrained=True)
        resnet.fc = nn.Identity()
    else:
        resnet = torchvision.models.resnet50(pretrained=False)  
        resnet.fc = nn.Identity()
        resnet.load_state_dict(torch.load(checkpoint))


    resnet = nn.DataParallel(resnet)
    resnet = resnet.to(device)
    print("Model created", flush=True)

    feats = []
    labels = []
    resnet = resnet.eval()
    for batch, (img) in enumerate(loader):
        img = img.to(device)
        
        with torch.no_grad():
            h_i = resnet(img)

        
        feats.append(h_i)
    
    feats = torch.cat(feats, dim=0)
    feats = feats.squeeze()
    feats = feats.cpu().numpy()
    
    print("Finished embedding. Feature array shape:", feats.shape)    
    
    return feats

This function takes feature vectors and will run k-means clustering with a given k value and will return the cluster labels for each feature vector. 

In [ ]:
def run_kmeans(feats, k=5, seed=51):
    sample_kmeans = KMeans(n_clusters=k, random_state=seed)
    labels = sample_kmeans.fit_predict(feats)
    return labels

This function creates maps based on the given k-cluster assignments of each patch. 

In [ ]:
def make_maps(labels, h5file, w, h, outfile, downsample = 20, tiledim = 256):
    
    f = h5py.File(h5file,'r')
    coords = np.asarray(f['coords'])


    mask = np.zeros((h,w,3))

    values = [ np.array(cm.Paired_r(i)[:3]) * 255 for i in set(labels) ]
    cmap = dict(zip(set(labels),values))

    for l,(x,y) in zip(labels,coords):
        mask[y:y+tiledim,x:x+tiledim,:] = cmap[l]

    image = Image.fromarray(mask.astype(np.uint8))
    size = image.size
    image = image.resize((int(size[0]/downsample),int(size[1]/downsample)))
    image.save(outfile)

    return

Here I do an example run through with a given case at k = 5

In [ ]:
k = 5
downsample = 70
h5file = '/sc/arion/projects/tauomics/PART_images/Hippocampus_AT8_stain/clam/256_allslides/patches/45505.h5'
slidepath = '/sc/arion/projects/tauomics/PART_images/Hippocampus_AT8_stain/45505.svs'
outfile = 'k5_byol_45505.png'
feats = make_features_to_np(slidepath, h5file, 256)
labels = run_kmeans(feats, k)

wsi = openslide.OpenSlide(slidepath)
w,h = wsi.dimensions

make_maps(labels,h5file,w,h,outfile, downsample)


Here we see the resulting map: 
![k5_byol_45505](https://github.com/john-mlr/deep-learning-project-2021/blob/main/k5_byol_45505.png?raw=true)

And the original slide image: 
![orig_at8_45505](https://github.com/john-mlr/deep-learning-project-2021/blob/main/orig_at8_45505.png?raw=true)

We can see how the clusters conform to the anatomical features and pathology. 

At k = 2, we can see that tau is a major discriminator in the feature space: 
![k2_byol_45505](https://github.com/john-mlr/deep-learning-project-2021/blob/main/k2_byol_45505.png?raw=true)





Here is K-means clustering on the same slide using feature vectors from a Resnet50 pretrained with Imagenet.

At k = 2:
![k2_pretrained_45505](https://github.com/john-mlr/deep-learning-project-2021/blob/main/k2_pretrained_45505.png?raw=true)

and at k = 5:
![k5_pretrained_45505](https://github.com/john-mlr/deep-learning-project-2021/blob/main/k5_pretrained_45505.png?raw=true)

While the feature space captured by Imagenet pretraining does an adequate job, it is not as fine grained and relevant to the pathology or anatomy as BYOL is. 

Here is an example with another slide. One with less tau burden. We find that the absence of tau results in more anatomy dependent features being brought out in the k-means clustering: 

Here is the original slide:
![orig_at8_45883](https://github.com/john-mlr/deep-learning-project-2021/blob/main/orig_at8_45883.png?raw=true)
Here is the clustering of BYOL features: 
BYOL at k = 2:
![k2_byol_45883](https://github.com/john-mlr/deep-learning-project-2021/blob/main/k2_byol_45883.png?raw=true)
BYOL at k = 5:
![k5_byol_45883](https://github.com/john-mlr/deep-learning-project-2021/blob/main/k5_byol_45883.png?raw=true)
Here is the clustering of Resnet50 pretrained with Imagenet:
pretrained at k = 2:
![k2_pretrained_45883](https://github.com/john-mlr/deep-learning-project-2021/blob/main/k2_pretrained_45883.png?raw=true)
pretrained at k = 5:
![k5_pretrained_45883](https://github.com/john-mlr/deep-learning-project-2021/blob/main/k5_pretrained_45883.png?raw=true)